# Creazione utenti

In [1]:
import os

import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine, text

In [2]:
drivername = "mysqlconnector"
user = "root"
pw = "ciao"
server = "127.0.0.1"
port = "3306"

db_name = "fabecare_db"

first_conn = f"mysql+{drivername}://{user}:{pw}@{server}:{port}/{db_name}"
engine = create_engine(first_conn)

In [3]:
filepath = "data/personale/Personale_Dataset.xlsx"
df = pd.read_excel(filepath, header = 0)

Nel blocco di codice seguente vengono estratte le informazioni relative agli utenti che avranno accesso al database.  
Per semplicità, tutte le password sono state impostate a "ciao".

In [4]:
users = []

for i in range(len(df['NOME'])):
    name = df['NOME'][i][0].lower()
    surname = df['COGNOME'][i].lower()
    if " " in surname:
        surname = surname.replace(" ", "_")
    user = f"{name}.{surname}"
    role = df['RUOLO'][i]
    users.append((user, role, df['REPARTO'][i]))

In [5]:
with engine.connect() as conn:
    for user in users:
        conn.execute(text(f"CREATE USER '{user[0]}'@localhost IDENTIFIED BY 'ciao'"))
        
        if user[1] == 'Medico' and user[2] == 'Terapia Intensiva':
            conn.execute(text(f"GRANT SELECT, INSERT, UPDATE ON fabecare_db.cure to '{user[0]}'@localhost"))
            conn.execute(text(f"GRANT SELECT, INSERT, UPDATE ON fabecare_db.parametri_vitali to '{user[0]}'@localhost"))
            conn.execute(text(f"GRANT SELECT, INSERT, UPDATE ON fabecare_db.pazienti to '{user[0]}'@localhost"))
            conn.execute(text(f"GRANT SELECT, INSERT, UPDATE ON fabecare_db.ricoveri to '{user[0]}'@localhost"))
            
        elif user[1] == 'Medico' and user[2] != 'Terapia Intensiva':
            conn.execute(text(f"GRANT SELECT, INSERT, UPDATE ON fabecare_db.cure to '{user[0]}'@localhost"))
            conn.execute(text(f"GRANT SELECT, INSERT, UPDATE ON fabecare_db.pazienti to '{user[0]}'@localhost"))
            conn.execute(text(f"GRANT SELECT, INSERT, UPDATE ON fabecare_db.ricoveri to '{user[0]}'@localhost"))
            
        elif user[1] == 'Infermiere':
            conn.execute(text(f"GRANT SELECT (IDP, NOME, COGNOME, DATA_NASCITA) ON fabecare_db.pazienti to '{user[0]}'@localhost"))
            conn.execute(text(f"GRANT SELECT (ID_PAZIENTE, ID_PERSONALE, PATOLOGIA, PRESCRIZIONI) ON fabecare_db.cure to '{user[0]}'@localhost"))
            conn.execute(text(f"GRANT SELECT ON fabecare_db.ricoveri to '{user[0]}'@localhost"))
            
        elif user[1] == 'OSS':
            pass
        
        elif user[1] == 'Pers. Tecnico':
            conn.execute(text(f"GRANT SELECT, DELETE ON fabecare_db.parametri_vitali to '{user[0]}'@localhost"))